# Adversarial Training

TJ Kim

11.29.21

#### Summary:
- Perform adversarial training in two different ways and analyze impact on inter-boundary distance of different variations of training
- Method 1: Train 100 rounds of regular FedEM/FedLocal (Same seed) --> 100 Rounds of adversarial Training
- Method 2: Train 200 rounds on benign model --> 200 rounds on adversarial model
- Examinations 1: Benign --> Adv trained 1 and 2 (Transferability and boundary distance)
- Examinations 2: Adv --> Adv trained 1 and 2 ("" "")

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


#### Import Libraries

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.Boundary_Transferer import *
from transfer_attacks.projected_gradient_descent import *

# da
from transfer_attacks.TA_utils import *

### Import FedEM Models

In [3]:
# Manually set argument parameters
args_ = Args()
args_.experiment = "femnist"
args_.method = "local"
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= 1
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/final/femnist/fig1_take2/Local_Benign/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_)

==> Clients initialization..
===> Building data iterators..


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:01<00:00, 449.02it/s]


===> Initializing clients..


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 719/719 [00:07<00:00, 98.67it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.151 | Train Acc: 1.472% |Test Loss: 4.147 | Test Acc: 1.434% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
clients[0].learners_ensemble.learners[0].model

FemnistCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=1024, out_features=408, bias=True)
  (output): Linear(in_features=408, out_features=62, bias=True)
)

In [5]:
# Import weights for aggregator
aggregator.load_state(args_.save_path)

# This is where the models are stored -- one for each mixture --> learner.model for nn
hypotheses = aggregator.global_learners_ensemble.learners

# obtain the state dict for each of the weights 
weights_h = []

for h in hypotheses:
    weights_h += [h.model.state_dict()]
    


In [7]:
weights = np.load("weights/cifar/21_09_28_first_transfers/train_client_weights.npy")
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#print(weights)

# Set model weights
model_weights = []
num_models = 7

for i in range(num_models):
    model_weights += [weights[i]]
    
    
# Generate the weights to test on as linear combinations of the model_weights
models_test = []

for (w0,w1,w2) in model_weights:
    # first make the model with empty weights
    new_model = copy.deepcopy(hypotheses[0].model)
    new_model.eval()
    new_weight_dict = copy.deepcopy(weights_h[0])
    for key in weights_h[0]:
        new_weight_dict[key] = w0*weights_h[0][key] # + w1*weights_h[1][key] + w2*weights_h[2][key]
    new_model.load_state_dict(new_weight_dict)
    models_test += [new_model]

In [51]:
clients[0].train_iterator.dataset.data[0]

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000

In [9]:
aggregator.clients[0].train_iterator.dataset.data[0].shape

torch.Size([28, 28])

In [10]:
aggregator.clients[0].train_iterator.dataset.targets

tensor([ 4, 38,  1,  3, 49,  8, 52, 15,  7,  2,  8,  4,  5,  0,  6,  5,  8,  0,
         2,  9,  4,  9, 28,  8,  3, 17, 23,  7,  9,  4,  5, 18,  3,  6,  0, 14,
         0,  9,  4, 32,  7,  2, 42, 27, 57, 41,  0,  9,  2,  1, 60,  5,  2,  7,
        35,  9, 44,  4,  1,  3,  3,  0, 43, 25,  4, 29,  3,  2,  8,  1,  6,  1,
         4,  3,  9,  0,  2,  1, 45, 22,  1,  0,  2,  3,  4,  1,  1,  6, 31, 26,
         5,  7,  0,  3, 48,  8, 16,  7, 21,  7,  7,  6, 53, 50, 40,  6,  6,  0,
        30,  6,  3, 33,  3,  4,  1,  5, 61,  9,  9,  4, 20,  1,  8, 37,  6,  2,
         8,  3, 11,  0,  7, 54, 10, 47,  5])

In [12]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
data_y = torch.tensor(data_y)

In [83]:
# Create dataloader from validation dataset that allows for diverse batch size
dataloader = Custom_Dataloader(data_x, data_y)
dataloader.x_data[0]

tensor([[[ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2

In [42]:
img1 = clients[0].train_iterator.dataset.data[0]
img1 = np.uint8(img1.numpy() * 255)
img1 = Image.fromarray(img1, mode='L')

img1
tt = ToTensor()
tt(img1)

tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
          1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,

In [35]:

forward = Normalize(mean=(0.1307,), std =(0.3081,))
normed = clients[0].train_iterator.dataset.transform(img1)

# torch.reshape(img1,[3,32,32])
# img1

In [37]:
normed

tensor([[[ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
         [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
           2.8215,  2.8215,  2.8215,  2.8215,  2

In [57]:
mean = torch.tensor([0.1307])
std = torch.tensor([0.3081])

unnormalize = Normalize((-mean / std).tolist(), (1.0 / std).tolist())
a = unnormalize(normed)
# a = a.transpose(0,1)
# a = a.transpose(1,2)
# a = a * 255
b = a.clone().detach().requires_grad_(True)#.type(torch.uint8)

In [60]:
b - tt(img1)

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,


In [64]:
def unnormalize_cifar10(normed):

    mean = torch.tensor([0.4914, 0.4822, 0.4465])
    std = torch.tensor([0.2023, 0.1994, 0.201])

    unnormalize = Normalize((-mean / std).tolist(), (1.0 / std).tolist())
    a = unnormalize(normed)
    a = a.transpose(0,1)
    a = a.transpose(1,2)
    a = a * 255
    b = a.clone().detach().requires_grad_(True).type(torch.uint8)
    
    return b

def unnormalize_femnist(normed):
    mean = torch.tensor([0.1307])
    std = torch.tensor([0.3081])
    
    unnormalize = Normalize((-mean / std).tolist(), (1.0 / std).tolist())
    a = unnormalize(normed)
    b = a.clone().detach().requires_grad_(True)
    return b

In [65]:
dataloader.x_data.shape

torch.Size([13516, 1, 28, 28])

In [66]:
data_xn = []
data_yn = []
rr = 1000

for i in range(rr):
    x = data_x[i]
    x_new = unnormalize_femnist(x)
    
    data_xn.append(x_new)

data_xn = torch.stack(data_xn)


In [87]:
data_xn.shape

torch.Size([1000, 1, 28, 28])

In [80]:
img1 = clients[0].train_iterator.dataset.data[1]
img1

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000